# Import Required Libraries
Import the necessary libraries, such as requests, for making API calls.

In [1]:
# Import the necessary library for making API calls
#%pip install requests
#%pip install pandas
#%pip install datetime
#%pip install openpyxl
#%pip install textblob
#%pip install transformers
#%pip install torch
import requests
import pandas as pd
import datetime as dt
import numpy as np
from textblob import TextBlob

# Define API Key
Define the API key

In [2]:
API_Eodhd = "66c0aeb1357b15.87356825"

# Creating a class with all the function


In [3]:
class EODHD_Functions:
    @staticmethod
    def Fetch_exchanges_list(api_token=API_Eodhd, filtered_columns=["Name", "Code", "Country", "Currency"]):
        url = f'https://eodhd.com/api/exchanges-list/?api_token={api_token}&fmt=json'
        response = requests.get(url).json()
        exchanges_df = pd.DataFrame(response)
        if filtered_columns:
            exchanges_df = exchanges_df[filtered_columns]
        return exchanges_df

    @staticmethod
    def Fetch_exchange_symbols(exchange_code, api_token=API_Eodhd, filtered_columns=None):
        url = f'https://eodhd.com/api/exchange-symbol-list/{exchange_code}?api_token={api_token}&fmt=json'
        response = requests.get(url).json()
        symbols_df = pd.DataFrame(response)
        if filtered_columns:
            symbols_df = symbols_df[filtered_columns]
        return symbols_df

    @staticmethod
    def Fetch_historical_price(stock, begin_date, end_date, period, api_token=API_Eodhd, columns=["adjusted_close"]):
        url = f'https://eodhd.com/api/eod/{stock}?from={begin_date}&to={end_date}&period={period}&api_token={api_token}&fmt=json'
        response = requests.get(url).json()
        pandas_df = pd.DataFrame(response)
        if columns:
            filtered_columns = ['date'] + columns
            pandas_df = pandas_df[filtered_columns]
        return pandas_df

    @staticmethod
    def Calculate_daily_returns(stock, begin_date, end_date, period, api_token=API_Eodhd):
        historical_data = EODHD_Functions.Fetch_historical_price(stock, begin_date, end_date, period, api_token, columns=["adjusted_close"])
        historical_data['daily_return'] = np.log(historical_data['adjusted_close'] / historical_data['adjusted_close'].shift(1))
        historical_data = historical_data.dropna()
        return historical_data[['date', 'daily_return']]

    @staticmethod
    def Fetch_news_data(stock, begin_date, end_date, offset=0, api_token=API_Eodhd):
        url = f'https://eodhd.com/api/news?s={stock}&offset={offset}&limit=1000&api_token={api_token}&from={begin_date}&to={end_date}&fmt=json'
        response = requests.get(url).json()
        news_df = pd.DataFrame(response)
        return news_df

    @staticmethod
    def Fetch_fundamentals(stock, api_token=API_Eodhd):
        url = f'https://eodhd.com/api/fundamentals/{stock}?api_token={api_token}&fmt=json'
        response = requests.get(url).json()
        fundamentals_df = pd.json_normalize(response)
        return fundamentals_df

    @staticmethod
    def Fetch_macro_indicator(country, indicator, api_token=API_Eodhd):
        url = f'https://eodhd.com/api/macro-indicator/{country}?indicator={indicator}&api_token={api_token}&fmt=json'
        response = requests.get(url).json()
        macro_df = pd.DataFrame(response)
        return macro_df

    @staticmethod
    def Fetch_intraday_data(stock, api_token=API_Eodhd):
        url = f'https://eodhd.com/api/intraday/{stock}?api_token={api_token}&fmt=json'
        response = requests.get(url)
        data = response.json()
        df = pd.DataFrame(data)
        if 'datetime' in df.columns:
            df['datetime'] = pd.to_datetime(df['datetime'])
        return df

    @staticmethod
    def Fetch_dividend_yield(tickers_df, exchange):
        combined_data = []
        full_tickers = [f"{ticker}.{exchange}" for ticker in tickers_df]
        for stock in full_tickers:
            try:
                url = f'https://eodhd.com/api/fundamentals/{stock}?api_token={API_Eodhd}&filter=Highlights::DividendYield&fmt=json'
                dividend_yield = requests.get(url).json()
                combined_data.append({"Ticker": stock, "DividendYield(%)": dividend_yield * 100})
            except Exception as e:
                print(f"Error fetching data for {stock}: {e}")
        combined_df = pd.DataFrame(combined_data)
        combined_df = combined_df.dropna(subset=["DividendYield(%)"])
        return combined_df

    @staticmethod
    def Fetch_dividend_share(tickers_df, exchange):
        combined_data = []
        full_tickers = [f"{ticker}.{exchange}" for ticker in tickers_df]
        for stock in full_tickers:
            try:
                url = f'https://eodhd.com/api/fundamentals/{stock}?api_token={API_Eodhd}&filter=Highlights::DividendShare&fmt=json'
                dividend_share = requests.get(url).json()
                combined_data.append({"Ticker": stock, "Dividendshare": dividend_share})
            except Exception as e:
                print(f"Error fetching data for {stock}: {e}")
        combined_df = pd.DataFrame(combined_data)
        combined_df = combined_df.dropna(subset=["Dividendshare"])
        return combined_df

    @staticmethod
    def Fetch_price_momentum(stock, days, api_token=API_Eodhd):
        try:
            end_date = dt.datetime.now()
            start_date = end_date - dt.timedelta(days=365)
            historical_data = EODHD_Functions.Fetch_historical_price(
                stock,
                start_date.strftime("%Y-%m-%d"),
                end_date.strftime("%Y-%m-%d"),
                "d",
                api_token,
                columns=["adjusted_close"]
            )
            historical_data = historical_data.sort_values(by="date")
            if len(historical_data) < days + 1:
                return None
            current_price = historical_data["adjusted_close"].iloc[-1]
            past_price = historical_data["adjusted_close"].iloc[-(days + 1)]
            log_return = np.log(current_price / past_price)
            return log_return
        except Exception as e:
            return None


In [4]:
# Function to fetch historical stock data from EODHD API

Client = EODHD_Functions()
exchanges = Client.Fetch_exchanges_list(api_token=API_Eodhd, filtered_columns=["Name", "Code", "Country", "Currency"])
print(exchanges.head())

                   Name Code Country Currency
0            USA Stocks   US     USA      USD
1       London Exchange  LSE      UK      GBP
2          NEO Exchange  NEO  Canada      CAD
3  TSX Venture Exchange    V  Canada      CAD
4      Toronto Exchange   TO  Canada      CAD


# Get list of tickets under an exchange
Define a function get list of tickers under an exchange

In [3]:
def fetch_exchanges_list(api_token=API_Eodhd, filtered_columns=["Name", "Code", "Country", "Currency"]):
    """
    Fetch the list of exchanges from the EODHD API and return it as a filtered Pandas DataFrame.

    Parameters:
        api_token (str): Your API token for authentication.
        filtered_columns (list): List of columns to include in the output DataFrame.

    Returns:
        pd.DataFrame: The exchange list as a Pandas DataFrame with selected columns.
    """
    url = f'https://eodhd.com/api/exchanges-list/?api_token={api_token}&fmt=json'
    response = requests.get(url).json()
    
    # Convert JSON to Pandas DataFrame
    exchanges_df = pd.DataFrame(response)
    
    # Filter columns if specified
    if filtered_columns:

        exchanges_df = exchanges_df[filtered_columns]
    
    return exchanges_df

def fetch_exchange_symbols(exchange_code, api_token=API_Eodhd, filtered_columns=None):
    """
    Fetch the symbol list for a given exchange from the EODHD API and return it as a Pandas DataFrame.

    Parameters:
        exchange_code (str): The code of the exchange (e.g., "US").
        api_token (str): Your API token for authentication.
        filtered_columns (list): List of columns to include in the output DataFrame (optional).

    Returns:
        pd.DataFrame: The symbol list as a Pandas DataFrame with selected columns.
    """
    url = f'https://eodhd.com/api/exchange-symbol-list/{exchange_code}?api_token={api_token}&fmt=json'
    response = requests.get(url).json()
    
    # Convert JSON to Pandas DataFrame
    symbols_df = pd.DataFrame(response)
    
    # Filter columns if specified
    if filtered_columns:
        symbols_df = symbols_df[filtered_columns]
    
    return symbols_df

'''tickers = fetch_exchange_symbols("AS")
print(tickers)'''

'tickers = fetch_exchange_symbols("AS")\nprint(tickers)'

# Get historical market data
uses a function to get historical market data

In [5]:
def Fetch_historical_price(stock, begin_date, end_date, period, api_token=API_Eodhd, columns=["adjusted_close"]):
    """
    Fetch stock data from the EODHD API and return it as a filtered Pandas DataFrame.

    Parameters:
        stock (str): The stock symbol (e.g., "MCD.US").
        begin_date (str): The start date in "YYYY-MM-DD" format.
        end_date (str): The end date in "YYYY-MM-DD" format.
        period (str): The period (e.g., "d" for daily).
        api_token (str): Your API token for authentication.
        columns (list): List of columns to filter (e.g., ['open', 'close','adjusted_close',volume]).

    Returns:
        pd.DataFrame: The filtered stock data as a Pandas DataFrame.
    """
    url = f'https://eodhd.com/api/eod/{stock}?from={begin_date}&to={end_date}&period={period}&api_token={api_token}&fmt=json'
    response = requests.get(url).json()
    
    # Convert JSON to Pandas DataFrame
    pandas_df = pd.DataFrame(response)
    
    # Filter columns if specified
    if columns:
       filtered_columns = ['date'] + columns
       pandas_df = pandas_df[filtered_columns]
    
    return pandas_df

# Example usage
ZYXI = Fetch_historical_price("ZYXI.US", "2023-01-01", "2023-10-01", "d" )
print(ZYXI.tail())

           date  adjusted_close
182  2023-09-25            8.41
183  2023-09-26            8.25
184  2023-09-27            8.23
185  2023-09-28            8.25
186  2023-09-29            8.00


# Get Daily returns
uses a calculate the daily returns of the stock

In [ ]:
def calculate_daily_returns(stock, begin_date, end_date, period, api_token=API_Eodhd):
    """
    Calculate the daily returns of a stock using the Fetch_historical_price function.

    Parameters:
        stock (str): The stock symbol (e.g., "ZYXI.US").
        begin_date (str): The start date in "YYYY-MM-DD" format.
        end_date (str): The end date in "YYYY-MM-DD" format.
        period (str): The period (e.g., "d" for daily).
        api_token (str): Your API token for authentication.

    Returns:
        pd.DataFrame: A DataFrame containing the date and daily returns.
    """
    # Fetch historical price data
    historical_data = Fetch_historical_price(stock, begin_date, end_date, period, api_token, columns=["adjusted_close"])
    
    # Calculate daily returns
    historical_data['daily_return'] = np.log(historical_data['adjusted_close'] / historical_data['adjusted_close'].shift(1))
    historical_data= historical_data.dropna()  # Drop the first row with NaN value
    
    return historical_data[['date', 'daily_return']]

# Example usage
'''daily_returns = calculate_daily_returns("ZYXI.US", "2023-01-01", "2023-10-01", "d")
print(daily_returns)'''

'daily_returns = calculate_daily_returns("ZYXI.US", "2023-01-01", "2023-10-01", "d")\nprint(daily_returns)'

# Getting financial news
Creates a function to retrive financial news

In [ ]:
def fetch_news_data(stock, tag ,begin_date, end_date, offset =0 , api_token =API_Eodhd):
    """
    Fetch news data for a given stock and return it as a Pandas DataFrame.

    Parameters:
        stock (str): The stock symbol (e.g., "AAPL.US").
        begin_date (str): The start date in "YYYY-MM-DD" format.
        end_date (str): The end date in "YYYY-MM-DD" format.
        tag (str): The tag to filter news (e.g., "balance sheet"). List of tags : 'balance sheet', 'capital employed', 'class action', 'company announcement', 'consensus eps estimate', 'consensus estimate', 'credit rating', 'discounted cash flow', 'dividend payments', 'earnings estimate', 'earnings growth', 'earnings per share', 'earnings release', 'earnings report', 'earnings results', 'earnings surprise', 'estimate revisions', 'european regulatory news', 'financial results', 'fourth quarter', 'free cash flow', 'future cash flows', 'growth rate', 'initial public offering', 'insider ownership', 'insider transactions', 'institutional investors', 'institutional ownership', 'intrinsic value', 'market research reports', 'net income', 'operating income', 'present value', 'press releases', 'price target', 'quarterly earnings', 'quarterly results', 'ratings', 'research analysis and reports', 'return on equity', 'revenue estimates', 'revenue growth', 'roce', 'roe', 'share price', 'shareholder rights', 'shareholder', 'shares outstanding', 'split', 'strong buy', 'total revenue', 'zacks investment research', 'zacks rank'
        offset (int): The offset for pagination.
        api_token (str): Your API token for authentication.

    Returns:
        pd.DataFrame: The news data as a Pandas DataFrame.
    """
    url = f'https://eodhd.com/api/news?s={stock}&t={tag}&offset={offset}&limit=1000&api_token={api_token}&from={begin_date}&to={end_date}&fmt=json'
    response = requests.get(url).json()
    
    # Convert JSON response to Pandas DataFrame
    news_df = pd.DataFrame(response)
    
    return news_df

def Fetch_news_data(stock, begin_date, end_date, offset =0 , api_token =API_Eodhd):
    """
    Fetch news data for a given stock and return it as a Pandas DataFrame.

    Parameters:
        stock (str): The stock symbol (e.g., "AAPL.US").
        begin_date (str): The start date in "YYYY-MM-DD" format.
        end_date (str): The end date in "YYYY-MM-DD" format.
        tag (str): The tag to filter news (e.g., "balance sheet"). List of tags : 'balance sheet', 'capital employed', 'class action', 'company announcement', 'consensus eps estimate', 'consensus estimate', 'credit rating', 'discounted cash flow', 'dividend payments', 'earnings estimate', 'earnings growth', 'earnings per share', 'earnings release', 'earnings report', 'earnings results', 'earnings surprise', 'estimate revisions', 'european regulatory news', 'financial results', 'fourth quarter', 'free cash flow', 'future cash flows', 'growth rate', 'initial public offering', 'insider ownership', 'insider transactions', 'institutional investors', 'institutional ownership', 'intrinsic value', 'market research reports', 'net income', 'operating income', 'present value', 'press releases', 'price target', 'quarterly earnings', 'quarterly results', 'ratings', 'research analysis and reports', 'return on equity', 'revenue estimates', 'revenue growth', 'roce', 'roe', 'share price', 'shareholder rights', 'shareholder', 'shares outstanding', 'split', 'strong buy', 'total revenue', 'zacks investment research', 'zacks rank'
        offset (int): The offset for pagination.
        api_token (str): Your API token for authentication.

    Returns:
        pd.DataFrame: The news data as a Pandas DataFrame.
    """
    url = f'https://eodhd.com/api/news?s={stock}&offset={offset}&limit=1000&api_token={api_token}&from={begin_date}&to={end_date}&fmt=json'
    response = requests.get(url).json()
    
    # Convert JSON response to Pandas DataFrame
    news_df = pd.DataFrame(response)
    
    return news_df

'''
# Example usage
stock = "AAPL.US"
begin_date = "2023-01-01"
end_date = "2023-10-01"
tag = "balance sheet"
offset = 0

news_df = fetch_news_data(stock, tag ,begin_date, end_date)'''

'\n# Example usage\nstock = "AAPL.US"\nbegin_date = "2023-01-01"\nend_date = "2023-10-01"\ntag = "balance sheet"\noffset = 0\n\nnews_df = fetch_news_data(stock, tag ,begin_date, end_date)'

# Getting fundamental data
Creating a function retrive fundamental data

In [7]:
def fetch_fundamentals(stock, api_token=API_Eodhd):
    """
    Fetch fundamental data for a given stock and return it as a Pandas DataFrame.

    Parameters:
        stock (str): The stock symbol (e.g., "AAPL.US").
        api_token (str): Your API token for authentication.

    Returns:
        pd.DataFrame: The fundamental data as a Pandas DataFrame.
    """
    url = f'https://eodhd.com/api/fundamentals/{stock}?api_token={api_token}&fmt=json'
    response = requests.get(url).json()
    
    # Flatten the JSON response and convert it to a Pandas DataFrame
    fundamentals_df = pd.json_normalize(response)
    
    return fundamentals_df

'''# Example usage
api_token = API_Eodhd  # Replace with your actual API token
stock = "AAPL.US"

fundamentals_df = fetch_fundamentals(stock, api_token)
print(fundamentals_df)'''

'# Example usage\napi_token = API_Eodhd  # Replace with your actual API token\nstock = "AAPL.US"\n\nfundamentals_df = fetch_fundamentals(stock, api_token)\nprint(fundamentals_df)'

# Getting macro economic data
Creating a function retrive macroeconomic data

In [8]:
def fetch_macro_indicator(country, indicator, api_token=API_Eodhd):
    """
    Fetch macroeconomic indicator data for a given country and return it as a Pandas DataFrame.

    Parameters:
        country (str): The country code (e.g., "USA").
        indicator (str): The macroeconomic indicator (e.g., "inflation_consumer_prices_annual"). List of indicators: https://eodhd.com/financial-apis/macroeconomics-data-and-macro-indicators-api
        api_token (str): Your API token for authentication.

    Returns:
        pd.DataFrame: The macroeconomic indicator data as a Pandas DataFrame.
    """
    url = f'https://eodhd.com/api/macro-indicator/{country}?indicator={indicator}&api_token={api_token}&fmt=json'
    response = requests.get(url).json()
    
    # Convert JSON response to Pandas DataFrame
    macro_df = pd.DataFrame(response)
    
    return macro_df

'''# Example usage
country = "USA"
indicator = "inflation_consumer_prices_annual"
macro_df = fetch_macro_indicator(country, indicator)
print(macro_df)'''


'# Example usage\ncountry = "USA"\nindicator = "inflation_consumer_prices_annual"\nmacro_df = fetch_macro_indicator(country, indicator)\nprint(macro_df)'

# Getting intraday stock data
Creating a function retrive stock data

In [9]:
def fetch_intraday_data(stock, api_token= API_Eodhd):
    """
    Fetch intraday data for a given stock from the EODHD API and convert it to a Pandas DataFrame.

    Parameters:
        stock (str): The stock symbol (e.g., "AAPL.US").
        api_token (str): Your API token for authentication.
        interval (str): The interval for intraday data (e.g., "1m", "5m", "15m", "1h"). Default is "1m".

    Returns:
        pd.DataFrame: A DataFrame containing the intraday data.
    """
    url = f'https://eodhd.com/api/intraday/{stock}?api_token={api_token}&fmt=json'
    response = requests.get(url)
    
    # Parse the JSON response
    data = response.json()
    
    # Convert JSON to Pandas DataFrame
    df = pd.DataFrame(data)
    
    # Ensure the 'datetime' column is in datetime format
    if 'datetime' in df.columns:
        df['datetime'] = pd.to_datetime(df['datetime'])
    
    return df

'''# Example usage
stock = "AAPL.US"

intraday_data = fetch_intraday_data(stock)
print(intraday_data.head())'''

'# Example usage\nstock = "AAPL.US"\n\nintraday_data = fetch_intraday_data(stock)\nprint(intraday_data.head())'

# Getting Dividend yield & Dividend Share
Creating a function to retrive dividend yield data and dividend share

In [10]:
def Fetch_dividend_yield(tickers_df, exchange):
    """
    Fetch dividend yields for all stocks in the tickers_df and combine them into a single DataFrame.

    Parameters:
        tickers_df (list or pd.Series): List of stock tickers (e.g., ["INGA", "ASML"]).
        exchange (str): Exchange code (e.g., "AS").

    Returns:
        pd.DataFrame: A DataFrame with two columns: 'Ticker' and 'DividendYield(%)'.
    """
    combined_data = []  # Initialize an empty list to store data
    full_tickers = [f"{ticker}.{exchange}" for ticker in tickers_df]

    for stock in full_tickers:
        try:
            # Fetch the dividend yield for the current stock
            url = f'https://eodhd.com/api/fundamentals/{stock}?api_token={API_Eodhd}&filter=Highlights::DividendYield&fmt=json'
            dividend_yield = requests.get(url).json()

                   
            # Append the stock and dividend yield to the list
            combined_data.append({"Ticker": stock, "DividendYield(%)": dividend_yield*100})
        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")

    # Convert the combined data into a DataFrame
    combined_df = pd.DataFrame(combined_data)

    # Drop rows with missing dividend yield values
    combined_df = combined_df.dropna(subset=["DividendYield(%)"])

    return combined_df


def Fetch_dividend_share(tickers_df, exchange):
    """
    Fetch dividend share for all stocks in the tickers_df and combine them into a single DataFrame.

    Parameters:
        tickers_df (list or pd.Series): List of stock tickers (e.g., ["INGA", "ASML"]).
        exchange (str): Exchange code (e.g., "AS").

    Returns:
        pd.DataFrame: A DataFrame with two columns: 'Ticker' and 'DividendYield(%)'.
    """
    combined_data = []  # Initialize an empty list to store data
    full_tickers = [f"{ticker}.{exchange}" for ticker in tickers_df]

    for stock in full_tickers:
        try:
            # Fetch the dividend share for the current stock
            url = f'https://eodhd.com/api/fundamentals/{stock}?api_token={API_Eodhd}&filter=Highlights::DividendShare&fmt=json'
            dividend_share = requests.get(url).json()

                   
            # Append the stock and dividend yield to the list
            combined_data.append({"Ticker": stock, "Dividendshare": dividend_share})
        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")

    # Convert the combined data into a DataFrame
    combined_df = pd.DataFrame(combined_data)

    # Drop rows with missing dividend yield values
    combined_df = combined_df.dropna(subset=["Dividendshare"])

    return combined_df


'''# Example usage
DY12 = Fetch_dividend_share(["INGA", "ASML"], "AS")
print(DY12)'''

'# Example usage\nDY12 = Fetch_dividend_share(["INGA", "ASML"], "AS")\nprint(DY12)'

# Creating a function to fetch live data
Creating a function to fetch live data

In [15]:
def fetch_intraday_data(stock, api_token= API_Eodhd):
    """
    Fetch intraday data for a given stock from the EODHD API and convert it to a Pandas DataFrame.

    Parameters:
        stock (str): The stock symbol (e.g., "AAPL.US").
        api_token (str): Your API token for authentication.
        interval (str): The interval for intraday data (e.g., "1m", "5m", "15m", "1h"). Default is "1m".

    Returns:
        pd.DataFrame: A DataFrame containing the intraday data.
    """
    url = f'https://eodhd.com/api/intraday/{stock}?api_token={api_token}&fmt=json'
    response = requests.get(url)
    
    # Parse the JSON response
    data = response.json()
    
    # Convert JSON to Pandas DataFrame
    df = pd.DataFrame(data)
    
    # Ensure the 'datetime' column is in datetime format
    if 'datetime' in df.columns:
        df['datetime'] = pd.to_datetime(df['datetime'])
    
    return df

# Example usage
stock = "AAPL.US"
intraday_data = fetch_intraday_data(stock)
print(intraday_data.tail())

       timestamp  gmtoffset            datetime        open        high  \
6473  1751053200          0 2025-06-27 19:40:00  201.115005  201.199996   
6474  1751053500          0 2025-06-27 19:45:00  201.035003  201.104995   
6475  1751053800          0 2025-06-27 19:50:00  200.979995  201.240005   
6476  1751054100          0 2025-06-27 19:55:00  200.979995  201.270004   
6477  1751054400          0 2025-06-27 20:00:00  201.080001  201.080001   

             low       close     volume  
6473  200.964996  201.030105   553068.0  
6474  200.880004  200.985000   587231.0  
6475  200.220001  201.134994  1642127.0  
6476  200.854995  201.100006  2640355.0  
6477  201.080001  201.080001        NaN  


# Creating price momentum function
Divides the latest stock price with the previous price specified (no. of days)


In [12]:
def fetch_price_momentum(stock, days, api_token=API_Eodhd):
    """
    Calculate the log return of a stock by dividing the current price with the price "days" ago.

    Parameters:
        stock (str): The stock symbol (e.g., "AAPL.US").
        days (int): The number of days ago to compare the price.
        api_token (str): Your API token for authentication.

    Returns:
        float: The log return of the stock, or None if an error occurs.
    """
    try:
        # Define the date range for fetching historical data (1 year of data)
        end_date = dt.datetime.now()
        start_date = end_date - dt.timedelta(days=365)  # Fetch 1 year of data

        # Fetch historical price data
        historical_data = Fetch_historical_price(
            stock,
            start_date.strftime("%Y-%m-%d"),
            end_date.strftime("%Y-%m-%d"),
            "d",
            api_token,
            columns=["adjusted_close"]
        )

        # Ensure the data is sorted by date
        historical_data = historical_data.sort_values(by="date")

        # Validate that there is enough data to calculate momentum
        if len(historical_data) < days + 1:
            return None

        # Get the current price and the price "days" ago
        current_price = historical_data["adjusted_close"].iloc[-1]
        past_price = historical_data["adjusted_close"].iloc[-(days + 1)]

        # Calculate the log return
        log_return = np.log(current_price / past_price)
        return log_return

    except Exception as e:
        return None

'''# Example usage
stock = "AAPL.US"
days = 25
log_return = fetch_price_momentum(stock, days)
print(log_return)'''

'# Example usage\nstock = "AAPL.US"\ndays = 25\nlog_return = fetch_price_momentum(stock, days)\nprint(log_return)'